Trial for scraping web 

In [1]:
#importing libraries
#scraping
from selenium import webdriver
from bs4 import BeautifulSoup
from threading import Timer
import sched
import requests
from queue import Queue
#data analyses
#import tensorflow as tf
#import cv2
from statistics import mean
import pandas as pd
from skimage import io

In [2]:
#INPUT
# @Link : vinted, subito, ...
# @Search: what I want to search

#OUTPUT
# @Products, prices, categories, images, links

global site, product 

global old_products_fifo

def web_scraper(Link,Search):
    
    global products, prices, categories, images, links, product_data

    products    = []    #List to store name of the product
    prices      = []    #List to store price of the product
    categories  = []    #List to store rating of the product
    images     = []    #List of Immages
    links       = []    #List of Links
    product_data = [] #Is a list containing lists [categories,products,prices,images,links]

    #select the web browser
    driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")

    #Will navigate to a page given by the URL forget
    if Link == 0: #Vinted
        baselink = 'https://www.vinted.it/catalog?search_text='
        filters = '&order=newest_first'
        driver.get(baselink + Search + filters)

    elif Link == 1: #Subito
        baselink = 'https://www.vinted.it/catalog?search_text='
        filters = '&order=newest_first'
        driver.get(baselink + Search +  filters)

    elif Link == 2: #Marketplace
        baselink = 'https://www.vinted.it/catalog?search_text='
        filters = '&order=newest_first'
        driver.get(baselink + Search +  filters)
    else:
        Err_case = 1
    
    #get source code 
    content = driver.page_source
    
    soup = BeautifulSoup(content)
    soup.prettify()

    #scraping the page
    for page in soup.findAll('div', attrs={"class" : "feed-grid"}):

        #scraping the boxes
        for box in page.findAll('div', attrs={"class" : "web_ui__ItemBox__box"}):

            #scraping the name the prize and the link for on a single box
            advertisement = []

            #CATEGORY
            category = box.find('div', attrs={'class':'web_ui__ItemBox__details'})
            categories.append(category.text)
            advertisement.append(category.text)
            #print(category.text)

            #PRICE
            price = box.find('div', attrs={'web_ui__ItemBox__title-content'})
            price_raw = price.text
            price_clean = price_raw.replace('\xa0', '')
            prices.append(price_clean)
            advertisement.append(price_clean)
            
            #LINK
            tmp = box.find('div', attrs={'web_ui__ItemBox__image-container'})
            link = tmp.find('a')
            links.append(link.get('href'))
            advertisement.append(link.get('href'))

            #SUBCATEGORY

            #IMMAGE and PRODUCT
            tmp = box.find('div', attrs={'web_ui__Image__image web_ui__Image__cover web_ui__Image__portrait web_ui__Image__scaled web_ui__Image__ratio'})
            image_link  = tmp.find('img')
            products.append(image_link.get('alt'))
            image = io.imread(image_link.get('src'))
            images.append(image)
            #print(image.get('alt'))
            #print(image.get('src'))
            advertisement.append(image_link.get('alt'))
            #advertisement.append(image) #Wrong
            product_data.append(advertisement) # contanins all the data grupped 

    #Reduce lenth
    products = products[0:10]
    prices = prices[0:10]
    categories = categories[0:10]
    links = links[0:10]
    images = images[0:10]
    product_data = product_data[0:10]
    #print(product_data)
    return products, prices, categories, links, images, product_data
        
def Web_init():
    #Define site and product 
    print("Insert the wanted site:")
    site = int(input("[0: Vinted ; 1:Subito ; 2 Marketplace]\n"))
    print(site)
    
    print("Insert the wanted product:")
    product = input("Insert the wanted product:\n")  
    print(product)

    return site, product

#def refresh():
#    driver.refresh()

def isnew(products_data, old_products):
    potential_products = []
    #print(products_data)
    #print("old product:")
    #print(old_products)
    for list in products_data:
        list[3]
        if list[3] not in old_products.queue:            
            potential_products.append(list)
            old_products.put(list[3])
    return potential_products

def scrape_prices_ebay(product_list):
    prices = {}
    for product in product_list:
        product = product[3]
        price = []
        #print(f"The product under analysis is: {product}")
        # format the search query
        search_query = product.replace(' ', '+')
        # send a request to eBay's search page
        response = requests.get(f'https://www.ebay.com/sch/i.html?_nkw={search_query}&LH_Complete=1&LH_Sold=1')
        soup = BeautifulSoup(response.content, 'html.parser')
        soup.prettify()
        # extract the first result from the search results 
        First = True
        #try:
        num = 0
        for result in soup.findAll('span', attrs={'class': 's-item__price'}):

            #print(f'Scraped string,{result}')
            if First == True:
                #print('First discrarded')
                First = False
                continue
            else:
                if "to" in result.text:
                    #print('Contains (to)')
                    continue
                else:
                    str = result.text
                    num = str.replace("$", "")
                    num = num.replace("€", "")
                    num = float(num.replace(",", ""))
                    num_round = round(num,2)
                    price.append(num_round)
                    #print(f'Single price,{price}')

        if len(price)>0:
            average_price = mean(price)
            max = average_price*10
            min = average_price/10

        for i in price:
            if i > max or i< min:
                price.remove(i)

        if len(price)>0:
            average_price = mean(price)


        #print(f'Average price,{average_price}')
        prices[product] = average_price
        
    print(f'Number of potential products:{len(product_list)}')
        #except:
        #    print(f'Error scrape_prices_ebay,{str}')
        #    prices[product] = 'Not found'
            
    return prices

def compare_prices(marketprices,potential_products):
    m_prices = list(marketprices.values())
    good_indeces = []
    bad_indeces = []
    count = 0

    for m_price in m_prices:
        product = potential_products[count]
        price = product[1]
        tmp = price.replace("$", "")
        tmp = tmp.replace("€", "")
        tmp = float(tmp.replace(",", ""))

        if tmp * 0.7 < m_price:
            good_indeces.append(count)
        else:
            bad_indeces.append(count)
            
        count +=1

    #Repo
    print(f'N. good product:{len(good_indeces)}')
    
    return good_indeces, bad_indeces


def main():

    # Create a scheduler instance
    sc = sched.scheduler()

    sc.enter(10, 1, web_scraper, (site, product) )
    # Run the scheduler
    sc.run()

    #Check new products
    potential_products = isnew(product_data, old_products_fifo)
    #print(f"New products inserted:{potential_products}")

    #scrape the new products prizes on the web
    marketprices = scrape_prices_ebay(potential_products)
    #print(f"Market price calculated:{marketprices}")
    
    #discard the object not founded
    #for marketprice in marketprices:
    #    if marketprice == 'Not found':
    #        marketprices.remove(marketprice)
    
    #print(f"Price:{marketprices}")
    
    #compare product price
    good_indeces, bad_indeces = compare_prices(marketprices,potential_products)
    
    #generate advertisement for good indeces
    m_prices = list(marketprices.values())
    for i in good_indeces:
        print(f"Value:{potential_products[i]}")
        print(f"Market price:{m_prices[i]}")
    


In [3]:
#Call the function

##WEB Init
[site, product] = Web_init()
web_scraper(site, product)

old_products = []
old_products_fifo = Queue(1000)

while True:

    try:
        main()

    except Exception as e:
        print(f'Error main: {e}')    